In [1]:
import tensorflow as  tf
from tensorflow import keras

In [2]:
!pip install -q -U keras-tuner

     |████████████████████████████████| 98 kB 5.8 MB/s 


In [3]:
import keras_tuner as kt

In [4]:
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

4431872/4422102 [==============================] - 0s 0us/step


In [5]:
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

Hypertuning: search space
no. of nodes in a hidden layer
range of values
Hypermodel

define a hyper model#
model builder function
subclass the hypermodel class of keras tuner API

predefined hyper model classes: HyperXception, HyperResNet

In [7]:
def model_builder(hp):
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(28, 28)))

  #tune the number of units/nodes in the first hidden layer
  #choose an optimal value between 32-512
  hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
  model.add(keras.layers.Dense(units=hp_units, activation='relu'))
  model.add(keras.layers.Dense(10))

  #tune the learning rate  of the optimizer
  #choose an optimal value from 0.01, 0.001, 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])
  return model

instantiate the tuner and perform hypertuning
4 tuners: RandomSearch, Hyperband, BayesianOptimization, Sklearn; Hyperband

1. Hypermodel: model_builder
2. the objective to optimize
3. maximum number of epochs to train

In [8]:
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='my_dir',
                     project_name='intro_to_kt')

In [9]:
#early stopping, to converge at a high performance
#overfitting
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [10]:
tuner.search(img_train, label_train, epochs=50, validation_split=0.2, callbacks=[stop_early])

#get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete.
The optimal number of units in the first layer is {best_hps.get('units')}
and the optimal learning rate for the optimizer is {best_hps.get('learning_rate')}.
""")

Trial 30 Complete [00h 00m 50s]
val_accuracy: 0.8666666746139526

Best val_accuracy So Far: 0.8917499780654907
Total elapsed time: 00h 15m 08s
INFO:tensorflow:Oracle triggered exit

The hyperparameter search is complete.
The optimal number of units in the first layer is 480
and the optimal learning rate for the optimizer is 0.001.



In [12]:
#train the model
#find the optimal number of epochs
model = tuner.hypermodel.build(best_hps)
history = model.fit(img_train, label_train, epochs=50, validation_split=0.2)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

Epoch 1/50
1500/1500 [==============================] - 9s 6ms/step - loss: 0.4942 - accuracy: 0.8246 - val_loss: 0.3924 - val_accuracy: 0.8589
Epoch 2/50
1500/1500 [==============================] - 7s 5ms/step - loss: 0.3662 - accuracy: 0.8666 - val_loss: 0.3542 - val_accuracy: 0.8721
Epoch 3/50
1500/1500 [==============================] - 8s 5ms/step - loss: 0.3298 - accuracy: 0.8787 - val_loss: 0.3330 - val_accuracy: 0.8783
Epoch 4/50
1500/1500 [==============================] - 8s 6ms/step - loss: 0.3067 - accuracy: 0.8850 - val_loss: 0.3233 - val_accuracy: 0.8846
Epoch 5/50
1500/1500 [==============================] - 8s 5ms/step - loss: 0.2846 - accuracy: 0.8936 - val_loss: 0.3204 - val_accuracy: 0.8857
Epoch 6/50
1500/1500 [==============================] - 8s 6ms/step - loss: 0.2707 - accuracy: 0.8982 - val_loss: 0.3178 - val_accuracy: 0.8854
Epoch 7/50
1500/1500 [==============================] - 8s 5ms/step - loss: 0.2554 - accuracy: 0.9046 - val_loss: 0.3566 - val_accuracy:

In [13]:
hypermodel = tuner.hypermodel.build(best_hps)

hypermodel.fit(img_train, label_train, epochs=best_epoch, validation_split=0.2)

Epoch 1/21
1500/1500 [==============================] - 8s 5ms/step - loss: 0.4943 - accuracy: 0.8266 - val_loss: 0.3960 - val_accuracy: 0.8624
Epoch 2/21
1500/1500 [==============================] - 8s 5ms/step - loss: 0.3694 - accuracy: 0.8656 - val_loss: 0.3724 - val_accuracy: 0.8644
Epoch 3/21
1500/1500 [==============================] - 8s 6ms/step - loss: 0.3298 - accuracy: 0.8791 - val_loss: 0.3360 - val_accuracy: 0.8795
Epoch 4/21
1500/1500 [==============================] - 8s 6ms/step - loss: 0.3079 - accuracy: 0.8878 - val_loss: 0.3218 - val_accuracy: 0.8838
Epoch 5/21
1500/1500 [==============================] - 8s 6ms/step - loss: 0.2882 - accuracy: 0.8942 - val_loss: 0.3453 - val_accuracy: 0.8778
Epoch 6/21
1500/1500 [==============================] - 8s 5ms/step - loss: 0.2718 - accuracy: 0.8988 - val_loss: 0.3267 - val_accuracy: 0.8851
Epoch 7/21
1500/1500 [==============================] - 8s 5ms/step - loss: 0.2570 - accuracy: 0.9044 - val_loss: 0.3275 - val_accuracy:

In [14]:
eval_result = hypermodel.evaluate(img_test, label_test)
print("[test loss, test accuracy]:", eval_result)

313/313 [==============================] - 1s 3ms/step - loss: 0.4465 - accuracy: 0.8771
[test loss, test accuracy]: [0.44648829102516174, 0.8770999908447266]
